In [1]:
from sbmfi.models.small_models import spiro

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [9]:
import math

def cartesian_to_polar_nd(coords):
    """
    Transforms Cartesian coordinates of a point in n-dimensional space to polar coordinates.

    Parameters:
        coords (list or tuple): Cartesian coordinates [x1, x2, ..., xn].

    Returns:
        tuple: Polar coordinates (r, theta1, theta2, ..., theta_{n-1}).
    """
    n = len(coords)
    r = math.sqrt(sum(c**2 for c in coords))
    if r == 0:
        return (0,) + (0,) * (n - 1)  # Special case: origin
    
    angles = []
    for i in range(n - 1):
        norm = math.sqrt(sum(c**2 for c in coords[i:]))
        angle = math.acos(coords[i] / norm) if norm != 0 else 0
        angles.append(angle)
    
    if coords[-1] < 0:  # Adjust the last angle to the full circle range
        angles[-1] = 2 * math.pi - angles[-1]

    return (r, *angles)


def polar_to_cartesian_nd(r, *angles):
    """
    Transforms polar coordinates of a point in n-dimensional space back to Cartesian coordinates.

    Parameters:
        r (float): Radius (distance from origin).
        angles (float): Angles (theta1, theta2, ..., theta_{n-1}).

    Returns:
        list: Cartesian coordinates [x1, x2, ..., xn].
    """
    n = len(angles) + 1
    coords = []
    for i in range(n):
        if i < n - 1:
            product = r
            for j in range(i):
                product *= math.sin(angles[j])
            product *= math.cos(angles[i])
        else:
            product = r
            for j in range(n - 1):
                product *= math.sin(angles[j])
        coords.append(product)
    return coords


In [10]:
cartesian_coords = (1, 1, 1)
polar_coords = cartesian_to_polar_nd(cartesian_coords)
print("Polar:", polar_coords)

back_to_cartesian = polar_to_cartesian_nd(*polar_coords)
print("Cartesian:", back_to_cartesian)


Polar: (1.7320508075688772, 0.9553166181245092, 0.7853981633974484)
Cartesian: [1.0, 0.9999999999999998, 1.0]


In [11]:
cartesian_coords = (1, 1, 1, 1)
polar_coords = cartesian_to_polar_nd(cartesian_coords)
print("Polar:", polar_coords)

back_to_cartesian = polar_to_cartesian_nd(*polar_coords)
print("Cartesian:", back_to_cartesian)

Polar: (2.0, 1.0471975511965979, 0.9553166181245092, 0.7853981633974484)
Cartesian: [0.9999999999999998, 1.0000000000000002, 1.0, 1.0000000000000002]


In [2]:
model, kwargs = spiro(backend='torch', build_simulator=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-27


C:\python_projects\sbmfi\venv\lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U3')) -> None

In [37]:
import torch
torch.Size((1,))


torch.Size([1])

In [34]:
import numpy as np

def multiple_try_metropolis_step(
    x,
    log_pi,          # function: log of target density, log_pi(state)
    sample_q,        # function: sample_q(current_state) -> propose new state
    log_q,           # function: log_q(from_state, to_state) -> log of proposal density
    K=5
):
    """
    Performs one Multiple-Try Metropolis step from current state x.

    Parameters
    ----------
    x : np.ndarray
        Current state of the chain.
    log_pi : callable
        log_pi(z) returns log of the target density at z (up to an additive constant).
    sample_q : callable
        sample_q(z) returns a single random draw from the proposal distribution q(z -> .).
    log_q : callable
        log_q(z_from, z_to) returns the log of the proposal density q(z_from -> z_to).
    K : int
        Number of multiple tries in each direction.

    Returns
    -------
    x_new : np.ndarray
        The next state of the chain after this MTM step.
    """
    # 1) Propose K candidates from x
    Y = [sample_q(x) for _ in range(K)]  # y_1, ..., y_K

    # 2) Compute forward weights w_i = π(y_i)*q(y_i->x)
    #    (We actually work in log-domain to avoid underflow)
    log_w = []
    for y in Y:
        log_wi = log_pi(y) + log_q(y, x)
        log_w.append(log_wi)
    # Convert log-weights to normal scale
    w = np.exp(log_w - np.max(log_w))  # subtract max for numerical stability
    W = np.sum(w)

    # 3) Choose one candidate y_i with probability proportional to w_i
    i = np.random.choice(K, p=w / W)
    y_chosen = Y[i]

    # 4) From y_i, propose K new points (including x as first one)
    Z = [x] + [sample_q(y_chosen) for _ in range(K - 1)]  # z_1 = x, plus K-1 others

    # 5) Compute reverse weights w'_j = π(z_j)*q(z_j->y_i)
    log_wprime = []
    for z_j in Z:
        log_wpj = log_pi(z_j) + log_q(z_j, y_chosen)
        log_wprime.append(log_wpj)
    wprime = np.exp(log_wprime - np.max(log_wprime))
    Wprime = np.sum(wprime)

    # 6) Probability that we "select x out of Z" in the reverse proposals
    #    (the analog of "accepting" in traditional MH)
    #    We do a discrete draw among Z with p'_j ~ w'_j / sum(w'_j)
    j = np.random.choice(K, p=wprime / Wprime)

    # 7) If j = 0 => we re-selected x (the first in Z). Accept the move to y_i.
    #    Otherwise, reject (remain at x).
    if j == 0:
        x_new = y_chosen
    else:
        x_new = x

    return x_new


def peskun_optimal_acceptance_ratio(
    x,
    y,
    Y,      # The K proposals from x: [y_1, ..., y_K]
    Z,      # The K proposals from y (including x): [z_1, ..., z_K], with z_1 = x
    log_pi,
    log_q
):
    """
    Computes the Peskun-optimal acceptance probability alpha(x->y)
    for a multiple-try Metropolis proposal.

    Parameters
    ----------
    x : np.ndarray
        Current state.
    y : np.ndarray
        Chosen proposal from x.
    Y : list of np.ndarray
        The K proposals from x, including y (or at least a set containing y).
    Z : list of np.ndarray
        The K proposals from y, with Z[0] = x and others random.
    log_pi : callable
        log_pi(z) = log of target density at z.
    log_q : callable
        log_q(z_from, z_to) = log of proposal density q(z_from->z_to).

    Returns
    -------
    alpha_xy : float
        The acceptance probability alpha(x->y).
    """
    # Weights for forward direction: w_i = pi(y_i)* q(y_i-> x)
    log_w_forward = np.array([log_pi(yi) + log_q(yi, x) for yi in Y])
    # Weights for reverse direction: w'_j = pi(z_j)* q(z_j-> y)
    log_w_reverse = np.array([log_pi(zj) + log_q(zj, y) for zj in Z])

    # Convert to numerical scale safely
    max_fw = np.max(log_w_forward)
    max_rv = np.max(log_w_reverse)

    w_forward = np.exp(log_w_forward - max_fw)
    w_reverse = np.exp(log_w_reverse - max_rv)

    W_forward = np.sum(w_forward)
    W_reverse = np.sum(w_reverse)

    # Identify log_pi(x), log_pi(y), log_q(x->y), log_q(y->x)
    lx = log_pi(x)
    ly = log_pi(y)
    lqxy = log_q(x, y)
    lqyx = log_q(y, x)

    # Peskun-optimal acceptance ratio:
    #     alpha = min(
    #         1,
    #         [ π(y)* q(y->x) * W_forward ] / [ π(x)* q(x->y) * W_reverse ]
    #     )
    # We'll compute that in log form to avoid underflow:
    log_num = ly + lqyx + np.log(W_forward)
    log_den = lx + lqxy + np.log(W_reverse)
    log_ratio = log_num - log_den
    print(log_ratio)
    alpha = min(1.0, np.exp(log_ratio))
    return alpha


# ----------------------------------------------------------------
# Example usage
if __name__ == "__main__":

    # Example log of target density: standard 2D Gaussian
    def log_pi_gaussian(z):
        return -0.5 * np.sum(z**2)  # ignoring constant terms for simplicity

    # Example proposal: isotropic Gaussian with variance = 1
    def sample_q(z):
        return z + np.random.randn(*z.shape)

    # Log of that proposal density q(x->y): Gaussian around x with identity cov
    def log_q(z_from, z_to):
        diff = z_to - z_from
        # log of N(0, I) = -0.5 * diff^2 - (D/2)*log(2π)
        # ignoring constant - (dim/2)*log(2π) for M-H ratio (cancels out anyway)
        return -0.5 * np.sum(diff**2)

    np.random.seed(8)

    # Current state
    x_current = np.array([0.0, 0.0])

    # Number of multiple tries
    K = 4

    # 1) Generate K proposals from x
    Y = [sample_q(x_current) for _ in range(K)]

    # 2) Suppose we pick the last one, y_chosen = Y[-1] (for demonstration)
    y_chosen = Y[-1]

    # 3) Generate K proposals from y_chosen, with x included as the first
    Z = [x_current] + [sample_q(y_chosen) for _ in range(K - 1)]

    # 4) Compute acceptance probability alpha(x->y_chosen)
    alpha_val = peskun_optimal_acceptance_ratio(
        x_current,
        y_chosen,
        Y,
        Z,
        log_pi_gaussian,
        log_q
    )

    print(f"Proposed move from x={x_current} to y={y_chosen}")
    print(f"Computed Peskun-optimal acceptance probability = {alpha_val:.4f}")

    # 5) You can accept/reject with this probability:
    a = np.random.rand()
    print(a, y_chosen)
    if a < alpha_val:
        x_new = y_chosen
    else:
        x_new = x_current

    print(f"Next state of the chain = {x_new}")


-4.079020067859467
Proposed move from x=[0. 0.] to y=[1.72783617 2.20455628]
Computed Peskun-optimal acceptance probability = 0.0169
0.06580838778721143 [1.72783617 2.20455628]
Next state of the chain = [0. 0.]


In [19]:
Y

[array([ 0.49671415, -0.1382643 ]),
 array([0.64768854, 1.52302986]),
 array([-0.23415337, -0.23413696]),
 array([1.57921282, 0.76743473])]

In [12]:
import numpy as np

def cartesian_to_polar_nd(coords):
    """
    Converts Cartesian coordinates to polar coordinates (angles only) in n-dimensional space.

    Parameters:
        coords (array-like): Cartesian coordinates [x1, x2, ..., xn].

    Returns:
        np.ndarray: Polar angles (theta1, theta2, ..., theta_{n-1}).
    """
    coords = np.asarray(coords)
    n = len(coords)
    
    # Calculate polar angles
    angles = []
    for i in range(n - 1):
        norm = np.linalg.norm(coords[i:])
        if norm == 0:
            angle = 0
        else:
            angle = np.arccos(coords[i] / norm)
        angles.append(angle)
    return np.array(angles)


def polar_to_cartesian_nd(angles):
    """
    Converts polar coordinates (angles only) back to Cartesian coordinates in n-dimensional space.

    Parameters:
        angles (array-like): Polar angles (theta1, theta2, ..., theta_{n-1}).

    Returns:
        np.ndarray: Cartesian coordinates [x1, x2, ..., xn].
    """
    angles = np.asarray(angles)
    n = len(angles) + 1
    
    coords = np.zeros(n)
    product = 1.0
    for i in range(n):
        if i < n - 1:
            coords[i] = product * np.cos(angles[i])
            product *= np.sin(angles[i])
        else:
            coords[i] = product
    return coords


In [13]:
# Cartesian to Polar
cartesian_coords = [1, 1, 1]
polar_angles = cartesian_to_polar_nd(cartesian_coords)
print("Polar Angles (3D):", polar_angles)

# Polar to Cartesian
reconstructed_cartesian = polar_to_cartesian_nd(polar_angles)
print("Reconstructed Cartesian (3D):", reconstructed_cartesian)


Polar Angles (3D): [0.95531662 0.78539816]
Reconstructed Cartesian (3D): [0.57735027 0.57735027 0.57735027]


In [14]:
# Cartesian to Polar
cartesian_coords = [1, 1, 1, 1]
polar_angles = cartesian_to_polar_nd(cartesian_coords)
print("Polar Angles (4D):", polar_angles)

# Polar to Cartesian
reconstructed_cartesian = polar_to_cartesian_nd(polar_angles)
print("Reconstructed Cartesian (4D):", reconstructed_cartesian)


Polar Angles (4D): [1.04719755 0.95531662 0.78539816]
Reconstructed Cartesian (4D): [0.5 0.5 0.5 0.5]


To compute the Jacobian of the inverse mapping from the ball \(\mathbb{B}^K(1)\) to the polytope \(\mathcal{F}^\ddagger\), let’s carefully outline the steps. Here's how the problem breaks down:

---

### 1. **Understanding the Mapping**

The given mapping can be expressed as:

1. **Forward Mapping (Polytope \(\to\) Ball):**

   \[
   \vec{v}^\ddagger \to \vec{v}^\mathbb{S} = [r, \vec{p}]^T
   \]

   Where:
   - \(r = \frac{d}{\alpha^{\max}}\) with \(d = \|\vec{v}^\ddagger\|_2\), and \(\alpha^{\max} = \min(\vec{\alpha} \mid \vec{\alpha} \geq 0)\).
   - \(\vec{p} = \text{polar}(\vec{c}) = \text{polar}(\frac{\vec{v}^\ddagger}{d})\) gives the spherical coordinates of the direction vector.

2. **Inverse Mapping (Ball \(\to\) Polytope):**

   \[
   \vec{v}^\mathbb{S} = [r, \vec{p}]^T \to \vec{v}^\ddagger
   \]

   Where:
   - \(\vec{c}\) is reconstructed from the polar coordinates \(\vec{p}\): \(\vec{c} = \text{cartesian}(\vec{p})\).
   - \(d = r \cdot \alpha^{\max}\), where \(\alpha^{\max}\) is determined by solving \(\alpha^{\max} = \min(\vec{\alpha} \mid \vec{\alpha} \geq 0)\), with:
     \[
     \vec{\alpha} = \vec{b}^\ddagger \oslash (\pmb{A}^\ddagger \cdot \vec{c}).
     \]
   - \(\vec{v}^\ddagger = d \cdot \vec{c}\).

---

### 2. **Inverse Mapping Function**

The inverse mapping is explicitly given as:
\[
\vec{v}^\ddagger = r \cdot \alpha^{\max} \cdot \vec{c},
\]
where:
1. \(\vec{c}\) depends on the polar coordinates \(\vec{p}\).
2. \(r\) is the radial distance.
3. \(\alpha^{\max}\) depends on \(\vec{c}\) and is defined as:
   \[
   \alpha^{\max} = \min(\vec{\alpha} \mid \vec{\alpha} \geq 0), \quad \vec{\alpha} = \vec{b}^\ddagger \oslash (\pmb{A}^\ddagger \cdot \vec{c}).
   \]

---

### 3. **Jacobian of the Inverse Mapping**

To compute the Jacobian \(J_{\text{inv}}\), we need to differentiate the inverse mapping \(\vec{v}^\ddagger = f_{\text{inv}}(r, \vec{p})\) with respect to \([r, \vec{p}]^T\).

#### Expression for \(\vec{v}^\ddagger\):
\[
\vec{v}^\ddagger = r \cdot \alpha^{\max} \cdot \vec{c}.
\]

The Jacobian will have contributions from the derivatives of:
1. \(\vec{c}\) (which depends on \(\vec{p}\)),
2. \(\alpha^{\max}\) (which depends on \(\vec{c}\) and indirectly on \(\vec{p}\)).

#### Components of the Jacobian:

1. **Derivative with respect to \(r\):**
   \[
   \frac{\partial \vec{v}^\ddagger}{\partial r} = \alpha^{\max} \cdot \vec{c}.
   \]

2. **Derivative with respect to \(\vec{p}\):**
   Using the product rule:
   \[
   \frac{\partial \vec{v}^\ddagger}{\partial \vec{p}} = r \cdot \frac{\partial (\alpha^{\max})}{\partial \vec{p}} \cdot \vec{c} + r \cdot \alpha^{\max} \cdot \frac{\partial \vec{c}}{\partial \vec{p}}.
   \]

---

### 4. **Breaking Down Derivatives**

#### \(\frac{\partial \vec{c}}{\partial \vec{p}}\):
This is the Jacobian of the transformation from polar to Cartesian coordinates. In \(K\)-dimensions:
\[
\vec{c} = \text{cartesian}(\vec{p}),
\]
and its derivatives are standard for polar-to-Cartesian transformations.

#### \(\frac{\partial \alpha^{\max}}{\partial \vec{p}}\):
Since:
\[
\alpha^{\max} = \min(\vec{\alpha} \mid \vec{\alpha} \geq 0),
\]
where:
\[
\vec{\alpha} = \vec{b}^\ddagger \oslash (\pmb{A}^\ddagger \cdot \vec{c}),
\]
the gradient of \(\alpha^{\max}\) depends on the closest constraint. Let \(\vec{\alpha}_i\) be the active constraint:
\[
\frac{\partial \alpha^{\max}}{\partial \vec{c}} = -\frac{\vec{b}^\ddagger_i}{(\pmb{A}^\ddagger \cdot \vec{c})_i^2} \cdot \pmb{A}^\ddagger_i.
\]

Using the chain rule:
\[
\frac{\partial \alpha^{\max}}{\partial \vec{p}} = \frac{\partial \alpha^{\max}}{\partial \vec{c}} \cdot \frac{\partial \vec{c}}{\partial \vec{p}}.
\]

---

### 5. **Final Jacobian**

The full Jacobian is:
\[
J_{\text{inv}} = \begin{bmatrix}
\frac{\partial \vec{v}^\ddagger}{\partial r} & \frac{\partial \vec{v}^\ddagger}{\partial \vec{p}}
\end{bmatrix},
\]
where:
\[
\frac{\partial \vec{v}^\ddagger}{\partial r} = \alpha^{\max} \cdot \vec{c},
\]
and:
\[
\frac{\partial \vec{v}^\ddagger}{\partial \vec{p}} = r \cdot \frac{\partial (\alpha^{\max})}{\partial \vec{p}} \cdot \vec{c} + r \cdot \alpha^{\max} \cdot \frac{\partial \vec{c}}{\partial \vec{p}}.
\]

This expression accounts for the dependencies on both \(r\) and \(\vec{p}\). Let me know if you want explicit forms for \(\frac{\partial \vec{c}}{\partial \vec{p}}\) in a specific dimension!

Let’s derive the expressions for the derivatives of \(\vec{v}^\ddagger = r \cdot \alpha^{\max} \cdot \vec{c}\) with respect to \(r\) and the polar coordinates \(\vec{p} = [\varphi_1, \dots, \varphi_{K-2}, \theta]\).

---

### 1. Expression for \(\vec{v}^\ddagger\):
The inverse mapping is:
\[
\vec{v}^\ddagger = r \cdot \alpha^{\max} \cdot \vec{c},
\]
where:
- \(r \in [0, 1]\) is the radial distance,
- \(\alpha^{\max} = \min\left(\vec{\alpha} \mid \vec{\alpha} \geq 0\right)\),
- \(\vec{\alpha} = \vec{b}^\ddagger \oslash (\pmb{A}^\ddagger \cdot \vec{c})\),
- \(\vec{c}\) is the Cartesian representation of the polar coordinates \(\vec{p}\):
  \[
  \vec{c} = \text{cartesian}(\vec{p}).
  \]

---

### 2. Derivative with Respect to \(r\):
To compute \(\frac{\partial \vec{v}^\ddagger}{\partial r}\), treat \(r\) as a scalar multiplier:
\[
\frac{\partial \vec{v}^\ddagger}{\partial r} = \alpha^{\max} \cdot \vec{c}.
\]

- Here, \(\alpha^{\max}\) and \(\vec{c}\) are functions of \(\vec{p}\), but they are independent of \(r\).
- Therefore, the derivative is straightforward.

---

### 3. Derivative with Respect to Polar Coordinates (\(\vec{p}\)):
The derivative with respect to polar coordinates \(\vec{p}\) is more complex because \(\vec{v}^\ddagger\) depends on \(\vec{p}\) both through \(\vec{c}\) and \(\alpha^{\max}\). Using the product rule:
\[
\frac{\partial \vec{v}^\ddagger}{\partial \vec{p}} = r \cdot \frac{\partial (\alpha^{\max})}{\partial \vec{p}} \cdot \vec{c} + r \cdot \alpha^{\max} \cdot \frac{\partial \vec{c}}{\partial \vec{p}}.
\]

---

#### 3.1. Derivative of \(\vec{c}\) with Respect to \(\vec{p}\):
Recall that \(\vec{c}\) is computed from polar coordinates:
\[
\vec{c} = \begin{bmatrix}
\cos(\varphi_1) \\
\sin(\varphi_1) \cos(\varphi_2) \\
\sin(\varphi_1) \sin(\varphi_2) \cos(\varphi_3) \\
\vdots \\
\prod_{i=1}^{K-2} \sin(\varphi_i) \cos(\theta) \\
\prod_{i=1}^{K-2} \sin(\varphi_i) \sin(\theta)
\end{bmatrix}.
\]

The derivative of \(\vec{c}\) with respect to the polar coordinates \(\vec{p} = [\varphi_1, \dots, \varphi_{K-2}, \theta]\) can be computed component-wise. For example:
- For \(\frac{\partial \vec{c}}{\partial \varphi_1}\):
  \[
  \frac{\partial c_1}{\partial \varphi_1} = -\sin(\varphi_1), \quad \frac{\partial c_2}{\partial \varphi_1} = \cos(\varphi_1) \cos(\varphi_2), \quad \text{etc.}
  \]
- For \(\frac{\partial \vec{c}}{\partial \theta}\), only the last two components depend on \(\theta\):
  \[
  \frac{\partial c_{K-1}}{\partial \theta} = -\prod_{i=1}^{K-2} \sin(\varphi_i) \sin(\theta), \quad \frac{\partial c_K}{\partial \theta} = \prod_{i=1}^{K-2} \sin(\varphi_i) \cos(\theta).
  \]

The full derivative \(\frac{\partial \vec{c}}{\partial \vec{p}}\) is the Jacobian matrix of the polar-to-Cartesian transformation.

---

#### 3.2. Derivative of \(\alpha^{\max}\) with Respect to \(\vec{p}\):
\(\alpha^{\max}\) depends on \(\vec{p}\) through \(\vec{c}\). The derivative is:
\[
\frac{\partial \alpha^{\max}}{\partial \vec{p}} = \frac{\partial \alpha^{\max}}{\partial \vec{c}} \cdot \frac{\partial \vec{c}}{\partial \vec{p}}.
\]

From the definition of \(\alpha^{\max}\):
\[
\alpha^{\max} = \min\left(\vec{\alpha} \mid \vec{\alpha} \geq 0\right), \quad \vec{\alpha} = \vec{b}^\ddagger \oslash (\pmb{A}^\ddagger \cdot \vec{c}),
\]
the gradient of \(\alpha^{\max}\) with respect to \(\vec{c}\) is:
\[
\frac{\partial \alpha^{\max}}{\partial \vec{c}} = -\frac{\vec{b}^\ddagger_i}{(\pmb{A}^\ddagger \cdot \vec{c})_i^2} \cdot \pmb{A}^\ddagger_i,
\]
where \(i\) corresponds to the active constraint (i.e., the constraint achieving \(\alpha^{\max}\)).

Thus:
\[
\frac{\partial \alpha^{\max}}{\partial \vec{p}} = -\frac{\vec{b}^\ddagger_i}{(\pmb{A}^\ddagger \cdot \vec{c})_i^2} \cdot \pmb{A}^\ddagger_i \cdot \frac{\partial \vec{c}}{\partial \vec{p}}.
\]

---

### 4. Final Expressions

1. **Derivative with respect to \(r\):**
   \[
   \frac{\partial \vec{v}^\ddagger}{\partial r} = \alpha^{\max} \cdot \vec{c}.
   \]

2. **Derivative with respect to \(\vec{p}\):**
   \[
   \frac{\partial \vec{v}^\ddagger}{\partial \vec{p}} = r \cdot \left( -\frac{\vec{b}^\ddagger_i}{(\pmb{A}^\ddagger \cdot \vec{c})_i^2} \cdot \pmb{A}^\ddagger_i \cdot \frac{\partial \vec{c}}{\partial \vec{p}} \right) \cdot \vec{c} + r \cdot \alpha^{\max} \cdot \frac{\partial \vec{c}}{\partial \vec{p}}.
   \]

These expressions combine the effects of both \(\alpha^{\max}\) and \(\vec{c}\) on \(\vec{v}^\ddagger\).

Certainly! Let’s break down the computation for a **single entry** in the Jacobian of \(\vec{v}^\ddagger = r \cdot \alpha^{\max} \cdot \vec{c}\) with respect to one polar coordinate, say \(\varphi_j\).

---

### 1. Expression for \(\vec{v}^\ddagger\):
The inverse mapping is:
\[
\vec{v}^\ddagger = r \cdot \alpha^{\max} \cdot \vec{c},
\]
where:
- \(r\) is the radial distance,
- \(\alpha^{\max}\) is the distance to the nearest constraint,
- \(\vec{c}\) is the Cartesian representation of the polar coordinates \(\vec{p}\).

---

### 2. Derivative of \(\vec{v}^\ddagger\) with respect to \(\varphi_j\):
Using the product rule:
\[
\frac{\partial \vec{v}^\ddagger}{\partial \varphi_j} = r \cdot \frac{\partial (\alpha^{\max})}{\partial \varphi_j} \cdot \vec{c} + r \cdot \alpha^{\max} \cdot \frac{\partial \vec{c}}{\partial \varphi_j}.
\]

#### Step 1: \(\frac{\partial \vec{c}}{\partial \varphi_j}\)
The Cartesian coordinates \(\vec{c} = [c_1, c_2, \dots, c_K]\) depend on the polar angles \([\varphi_1, \dots, \varphi_{K-2}, \theta]\). Each component \(c_k\) can be written as:
\[
c_k = \prod_{i=1}^{k-1} \sin(\varphi_i) \cdot 
\begin{cases} 
\cos(\varphi_k) & \text{if } k < K, \\
\cos(\theta) & \text{if } k = K-1, \\
\sin(\theta) & \text{if } k = K.
\end{cases}
\]

The derivative of \(c_k\) with respect to \(\varphi_j\) depends on \(j\):
- If \(j < k\), \(c_k\) includes \(\sin(\varphi_j)\), so:
  \[
  \frac{\partial c_k}{\partial \varphi_j} = \prod_{i=1}^{j-1} \sin(\varphi_i) \cdot \cos(\varphi_j) \cdot \prod_{i=j+1}^{k-1} \sin(\varphi_i) \cdot (\text{rest of } c_k).
  \]
- If \(j = k\), \(c_k = \cos(\varphi_k)\), so:
  \[
  \frac{\partial c_k}{\partial \varphi_j} = -\prod_{i=1}^{j-1} \sin(\varphi_i) \cdot \sin(\varphi_j).
  \]
- If \(j > k\), \(c_k\) is independent of \(\varphi_j\), so:
  \[
  \frac{\partial c_k}{\partial \varphi_j} = 0.
  \]

---

#### Step 2: \(\frac{\partial (\alpha^{\max})}{\partial \varphi_j}\)
The distance \(\alpha^{\max}\) depends on \(\vec{c}\):
\[
\alpha^{\max} = \min\left(\vec{\alpha} \mid \vec{\alpha} \geq 0\right), \quad \vec{\alpha} = \vec{b}^\ddagger \oslash (\pmb{A}^\ddagger \cdot \vec{c}).
\]

The derivative of \(\alpha^{\max}\) with respect to \(\varphi_j\) is:
\[
\frac{\partial \alpha^{\max}}{\partial \varphi_j} = -\frac{b^\ddagger_i}{\left(\pmb{A}^\ddagger \cdot \vec{c}\right)_i^2} \cdot \pmb{A}^\ddagger_{i, :} \cdot \frac{\partial \vec{c}}{\partial \varphi_j},
\]
where \(i\) is the index of the active constraint (the one that achieves \(\alpha^{\max}\)).

---

#### Step 3: Putting it Together
Combining both terms:
\[
\frac{\partial v^\ddagger_k}{\partial \varphi_j} = r \cdot \left[ -\frac{b^\ddagger_i}{\left(\pmb{A}^\ddagger \cdot \vec{c}\right)_i^2} \cdot A^\ddagger_{i, k} \cdot \frac{\partial c_k}{\partial \varphi_j} \right] + r \cdot \alpha^{\max} \cdot \frac{\partial c_k}{\partial \varphi_j}.
\]

Here:
1. \(\frac{\partial c_k}{\partial \varphi_j}\) depends on whether \(j < k\), \(j = k\), or \(j > k\), as shown above.
2. The first term includes the dependence of \(\alpha^{\max}\) on \(\varphi_j\) and involves the constraint matrix \(\pmb{A}^\ddagger\).

---

### 4. Single Entry Example
If \(k = 3\) and \(j = 2\), the derivative is:
\[
\frac{\partial v^\ddagger_3}{\partial \varphi_2} = r \cdot \left[ -\frac{b^\ddagger_i}{\left(\pmb{A}^\ddagger \cdot \vec{c}\right)_i^2} \cdot A^\ddagger_{i, 3} \cdot \frac{\partial c_3}{\partial \varphi_2} \right] + r \cdot \alpha^{\max} \cdot \frac{\partial c_3}{\partial \varphi_2}.
\]

Here:
- \(\frac{\partial c_3}{\partial \varphi_2}\) is determined from the Cartesian-to-polar relationship.

If you'd like, I can further simplify it for specific values or provide Python code! Let me know!

In [50]:
import numpy as np

def tjelmeland_barker_transition_matrix(
    x_set,
    log_pi,
    log_q,
    force_jump=False
):
    """
    Construct Tjelmeland's multi-proposal Barker transition kernel 
    for x_set = [x_0, x_1, ..., x_K],
    allowing (but not guaranteeing) a nonzero self-transition if force_jump=False.

    Barker's pairwise function:
        phi(u,v) = u / (u + v)
    ensures detailed balance if each row is normalized 
    so that T[i,j] = phi(w[i,j], w[j,i]) / sum_k phi(w[i,k], w[k,i]) for k != i.

    Parameters
    ----------
    x_set : list of np.ndarray
        (K+1) states: x_set[0] is 'current' state, x_set[1..K] are proposals.
    log_pi : callable
        log_pi(x) -> float, log of target density at x (up to a constant).
    log_q : callable
        log_q(x_from, x_to) -> float, log of proposal PDF q(x_from-> x_to).
    force_jump : bool
        If True, set T[i,i] = 0 explicitly. 
        If False, set T[i,i] = leftover so that each row sums to 1 
        (but often leftover is zero anyway, given how Barker’s rule sums up).

    Returns
    -------
    T : (K+1) x (K+1) np.ndarray
        Row-stochastic matrix. T[i,j] = probability of moving from x_i to x_j.
    """
    def barker_phi(u, v):
        # phi(u,v) = u/(u+v), with safe handling for edge cases
        if u <= 0 and v <= 0:
            return 0.0
        return u / (u + v)

    n = len(x_set)
    T = np.zeros((n, n), dtype=float)

    # 1) Build the log-weights w[i->j] = log [ pi(x_j)*q(x_j-> x_i) ]
    log_w = np.full((n, n), -np.inf)
    for i in range(n):
        for j in range(n):
            if i != j:
                val = log_pi(x_set[j]) + log_q(x_set[j], x_set[i])
                log_w[i,j] = val
            else:
                log_w[i,j] = -np.inf

    # Exponentiate rows stably
    w = np.zeros((n, n), dtype=float)
    for i in range(n):
        row_vals = log_w[i, :]
        max_val = np.max(row_vals)
        if np.isfinite(max_val):
            row_exp = np.exp(row_vals - max_val)
            w[i,:] = row_exp
        # else remains zeros

    # 2) Compute row i, T[i,j] = phi(w[i,j], w[j,i]) / sum_{k != i} phi(...).
    for i in range(n):
        phi_vals = np.zeros(n, dtype=float)
        for k in range(n):
            if k != i:
                phi_vals[k] = barker_phi(w[i,k], w[k,i])

        denom = np.sum(phi_vals)
        if denom > 0:
            # Off-diagonal probabilities
            for j in range(n):
                if j != i:
                    T[i,j] = phi_vals[j] / denom
            if force_jump:
                # Force zero self-transition
                T[i,i] = 0.0
            else:
                # Let leftover = 1 - sum_{j!=i} T[i,j].
                # Usually sum_{j!=i} T[i,j] = 1.0 => leftover=0
                leftover = 1.0 - np.sum(T[i, :])
                # If leftover < 0, that means sum_{j!=i} T[i,j] > 1 => numerical or Barker’s rule
                # We'll just clamp to 0 to keep row-stochastic
                T[i,i] = max(0.0, leftover)
                # Then re-normalize the row so it sums to 1
                row_sum = np.sum(T[i,:])
                if row_sum > 0:
                    T[i,:] /= row_sum
                else:
                    # degenerate: everything is 0 => stay put
                    T[i,i] = 1.0
        else:
            # If no valid moves => remain with prob 1
            T[i,i] = 1.0

    return T

def sample_from_transition(i_current, T):
    """Sample next index from row i_current of matrix T."""
    p = T[i_current, :]
    i_next = np.random.choice(len(p), p=p)
    return i_next


# ----------------------------------------------------------------
# DEMO
if __name__ == "__main__":
    # Example target: 2D standard Gaussian
    def log_pi_gaussian(z):
        return -0.5 * np.sum(z**2)  # ignoring additive constants

    # Isotropic Gaussian proposal
    sigma = 1.0
    def sample_q(x):
        return x + sigma * np.random.randn(*x.shape)
    def log_q(x_from, x_to):
        diff = x_to - x_from
        return -0.5 * np.sum(diff**2) / (sigma**2)

    np.random.seed(1)
    x0 = np.array([0.0, 0.0])

    K = 3
    # Suppose we propose K states from x0
    proposals = [sample_q(x0) for _ in range(K)]
    x_set = [x0] + proposals

    # Build T with force_jump=False
    T = tjelmeland_barker_transition_matrix(
        x_set, log_pi_gaussian, log_q, force_jump=False
    )

    print("x_set:")
    for i, s in enumerate(x_set):
        print(f"  i={i} => {s}")

    print("\nBarker Transition Matrix T (force_jump=False):")
    np.set_printoptions(precision=3, suppress=True)
    print(T)

    row_sums = T.sum(axis=1)
    print("\nRow sums (should be 1):", row_sums)

    # Sample from row 0
    i_next = sample_from_transition(0, T)
    print(f"\nSampled next index from row 0 => i_next={i_next}, x_next={x_set[i_next]}")


x_set:
  i=0 => [0. 0.]
  i=1 => [ 1.624 -0.612]
  i=2 => [-0.528 -1.073]
  i=3 => [ 0.865 -2.302]

Barker Transition Matrix T (force_jump=False):
[[0.    0.248 0.727 0.025]
 [0.477 0.    0.477 0.046]
 [0.727 0.248 0.    0.025]
 [0.34  0.319 0.34  0.   ]]

Row sums (should be 1): [1. 1. 1. 1.]

Sampled next index from row 0 => i_next=2, x_next=[-0.528 -1.073]


In [39]:
import numpy as np

def tjelmeland_peskun_transition_matrix(x_set, log_pi, log_q):
    """
    Construct Tjelmeland's Peskun transition matrix for states in x_set = [x_0, x_1, ..., x_K].

    Parameters
    ----------
    x_set : list of np.ndarray
        List of (K+1) states: x_set[0] is the 'current' state, x_set[1..K] are proposals.
    log_pi : callable
        log_pi(x) -> float, returns log of target density at x.
    log_q : callable
        log_q(x_from, x_to) -> float, returns log of proposal PDF q(x_from-> x_to).
    Returns
    -------
    T : np.ndarray, shape (K+1, K+1)
        Row-stochastic transition matrix satisfying detailed balance w.r.t. pi.
        T[i,j] = probability of moving from state i to state j.
    """
    n = len(x_set)  # K+1 states
    T = np.zeros((n, n), dtype=float)

    # Compute unnormalized weights w_{i->j}
    # w_{i->j} = pi(x_j) * q(x_j -> x_i), for i != j
    # We'll work in the log domain for numerical stability:
    log_w = np.full((n, n), -np.inf)

    for i in range(n):
        for j in range(n):
            if i != j:
                # log_w[i,j] = log_pi(x_j) + log_q(x_j, x_i)
                log_w[i,j] = log_pi(x_set[j]) + log_q(x_set[j], x_set[i])
            else:
                # log_w[i,j] = -np.inf  # i == j => no self-transition weight
                log_w[i,j] = log_pi(x_set[j]) + log_q(x_set[j], x_set[i])  # i == j => no self-transition weight

    # Now, for each row i, we normalize over j != i to get a probability distribution
    for i in range(n):
        # We want to exponentiate log_w[i,*] in a stable way
        row_log_values = log_w[i, :]  # shape (n,)
        max_val = np.max(row_log_values)
        if np.isinf(max_val):
            # Degenerate case, e.g. if all w_{i->j} = 0 for j != i
            # We'll handle by letting T[i,i] = 1 or something
            T[i,i] = 1.0
        else:
            # Exponentiate shifted values
            row_exp = np.exp(row_log_values - max_val)  # shift for stability
            row_sum = np.sum(row_exp)
            if row_sum > 0:
                T[i, :] = row_exp / row_sum
            else:
                # If row_sum=0, fallback
                T[i, i] = 1.0

    return T


def sample_from_transition(i_current, T):
    """
    Given a row-stochastic matrix T and a current row index i_current,
    sample the next index from T[i_current, :].

    Parameters
    ----------
    i_current : int
        The current index in {0, 1, ..., K}.
    T : np.ndarray
        Transition matrix, shape (K+1, K+1).

    Returns
    -------
    i_next : int
        The chosen next index, drawn with probabilities T[i_current, :].
    """
    p = T[i_current, :]
    i_next = np.random.choice(len(p), p=p)
    return i_next


# ----------------------------------------------------------------
# A DEMO USAGE
if __name__ == "__main__":
    # Example: 2D standard Gaussian target distribution (log-scale)
    def log_pi_gaussian(z):
        return -0.5 * np.sum(z**2)  # ignoring normalizing constants

    # Gaussian proposal: q(x->y) ~ N(x, sigma^2 I)
    sigma = 1.0
    def sample_q(x):
        return x + sigma * np.random.randn(*x.shape)

    def log_q(x_from, x_to):
        diff = x_to - x_from
        # log of N(0, sigma^2 I) ignoring constant dims => -|diff|^2/(2 sigma^2)
        return -0.5 * np.sum(diff**2) / (sigma**2)

    # We want to form x_set = [x_0, x_1, ..., x_K].
    K = 3
    x0 = np.array([0.0, 0.0])  # current state
    proposals = [sample_q(x0) for _ in range(K)]
    x_set = [x0] + proposals

    # Build Tjelmeland's Peskun transition matrix
    T = tjelmeland_peskun_transition_matrix(x_set, log_pi_gaussian, log_q)

    # Current index is i=0 => corresponds to x0
    i_current = 0
    i_next = sample_from_transition(i_current, T)

    print("States (K+1):")
    for i, xx in enumerate(x_set):
        print(f" i={i} => {xx}")

    print("\nTransition matrix T (rows sum to 1):")
    np.set_printoptions(precision=3, suppress=True)
    print(T)

    print(f"\nFrom i_current=0 (i.e., {x0}), we sample i_next={i_next}")
    print(f"   => next state in x_set is {x_set[i_next]}")


States (K+1):
 i=0 => [0. 0.]
 i=1 => [-0.066 -0.209]
 i=2 => [ 1.347 -0.607]
 i=3 => [-0.174  0.424]

Transition matrix T (rows sum to 1):
[[0.348 0.331 0.039 0.282]
 [0.349 0.349 0.041 0.262]
 [0.287 0.284 0.287 0.142]
 [0.339 0.299 0.023 0.339]]

From i_current=0 (i.e., [0. 0.]), we sample i_next=1
   => next state in x_set is [-0.066 -0.209]
